In [1]:
from datetime import datetime

def log_progress(message):
    with open("code_log.txt", "a") as log_file:
        time_stamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log_file.write(f"[{time_stamp}] {message}\n")


In [2]:
log_progress("Script started")


In [9]:
import pandas as pd

url = "https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks"
tables = pd.read_html(url)

print(f"Total tables found: {len(tables)}")

for i, table in enumerate(tables):
    print(f"\nTable {i}: Columns --> {table.columns}")


Total tables found: 4

Table 0: Columns --> Index([0, 1, 2], dtype='int64')

Table 1: Columns --> Index(['Rank', 'Bank name', 'Market cap (US$ billion)'], dtype='object')

Table 2: Columns --> Index(['Rank', 'Bank name', 'Total assets (2022) (US$ billion)'], dtype='object')

Table 3: Columns --> Index(['Rank', 'Country', 'Number'], dtype='object')


In [12]:
def extract():
    url = "https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks"
    tables = pd.read_html(url)

    # Look for the table with correct columns
    for table in tables:
        if 'Market cap (US$ billion)' in table.columns:
            df = table[['Bank name', 'Market cap (US$ billion)']].copy()
            df.rename(columns={
                'Bank name': 'Name',
                'Market cap (US$ billion)': 'MC_USD_Billion'
            }, inplace=True)
            df = df.head(10)
            log_progress("Data extraction completed")
            return df


In [13]:
df = extract()
print(df)


                                      Name  MC_USD_Billion
0                           JPMorgan Chase          432.92
1                          Bank of America          231.52
2  Industrial and Commercial Bank of China          194.56
3               Agricultural Bank of China          160.68
4                                HDFC Bank          157.91
5                              Wells Fargo          155.87
6                        HSBC Holdings PLC          148.90
7                           Morgan Stanley          140.83
8                  China Construction Bank          139.82
9                            Bank of China          136.81


In [5]:
pip install lxml

   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.5/4.0 MB 1.9 MB/s eta 0:00:02
   ------- -------------------------------- 0.8/4.0 MB 1.9 MB/s eta 0:00:02
   ------------- -------------------------- 1.3/4.0 MB 2.2 MB/s eta 0:00:02
   -------------------- ------------------- 2.1/4.0 MB 2.3 MB/s eta 0:00:01
   -------------------------- ------------- 2.6/4.0 MB 2.5 MB/s eta 0:00:01
   --------------------------------- ------ 3.4/4.0 MB 2.7 MB/s eta 0:00:01
   ---------------------------------------- 4.0/4.0 MB 2.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
def transform(df):
    exchange_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv"
    exchange_df = pd.read_csv(exchange_url)
    
    # Convert exchange dataframe to a dictionary: {'GBP': rate, ...}
    rates = dict(zip(exchange_df['Currency'], exchange_df['Rate']))
    
    # Add new columns
    df['MC_GBP_Billion'] = round(df['MC_USD_Billion'] * rates['GBP'], 2)
    df['MC_EUR_Billion'] = round(df['MC_USD_Billion'] * rates['EUR'], 2)
    df['MC_INR_Billion'] = round(df['MC_USD_Billion'] * rates['INR'], 2)

    log_progress("Data transformation completed")
    return df


In [15]:
df_transformed = transform(df)
print(df_transformed)


                                      Name  MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase          432.92          346.34   
1                          Bank of America          231.52          185.22   
2  Industrial and Commercial Bank of China          194.56          155.65   
3               Agricultural Bank of China          160.68          128.54   
4                                HDFC Bank          157.91          126.33   
5                              Wells Fargo          155.87          124.70   
6                        HSBC Holdings PLC          148.90          119.12   
7                           Morgan Stanley          140.83          112.66   
8                  China Construction Bank          139.82          111.86   
9                            Bank of China          136.81          109.45   

   MC_EUR_Billion  MC_INR_Billion  
0          402.62        35910.71  
1          215.31        19204.58  
2          180.94        16138.75

In [16]:
def load_to_csv(df, filename="Largest_banks_data.csv"):
    df.to_csv(filename, index=False)
    log_progress("Data loaded to CSV")


In [17]:
load_to_csv(df_transformed)


In [18]:
import sqlite3

def load_to_db(df, db_name="Banks.db"):
    conn = sqlite3.connect(db_name)
    df.to_sql("Largest_banks", conn, if_exists="replace", index=False)
    conn.close()
    log_progress("Data loaded to Database")


In [19]:
load_to_db(df_transformed)


In [20]:
def run_queries(db_name="Banks.db"):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    log_progress("Running SQL Queries")

    # Query 1: Top 5 banks by MC_INR_Billion
    query1 = "SELECT Name, MC_INR_Billion FROM Largest_banks ORDER BY MC_INR_Billion DESC LIMIT 5"
    results = cursor.execute(query1).fetchall()
    print("Top 5 Banks by Market Cap in INR:")
    for row in results:
        print(row)

    # Query 2: Average Market Cap in USD
    query2 = "SELECT AVG(MC_USD_Billion) FROM Largest_banks"
    avg_usd = cursor.execute(query2).fetchone()[0]
    print(f"\nAverage Market Cap in USD: {round(avg_usd, 2)} Billion")

    conn.close()
    log_progress("SQL Queries executed")


In [21]:
run_queries()


Top 5 Banks by Market Cap in INR:
('JPMorgan Chase', 35910.71)
('Bank of America', 19204.58)
('Industrial and Commercial Bank of China', 16138.75)
('Agricultural Bank of China', 13328.41)
('HDFC Bank', 13098.63)

Average Market Cap in USD: 189.98 Billion
